In [ ]:
//#r "./../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
#r "BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using System.IO;
using BoSSS.Foundation.IO;
using ilPSP.Tracing;
using BoSSS.Application.BoSSSpad;
using BoSSS.Solution.Gnuplot;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
// Is used at Jenkins to generate individual names (for output .json)
string dbname = System.Environment.GetEnvironmentVariable("DATABASE_NAME");
string buildname = System.Environment.GetEnvironmentVariable("BUILD_DISPLAY_NAME");
string buildNr = System.Environment.GetEnvironmentVariable("BUILD_NUMBER");
//defaults
buildname = String.IsNullOrEmpty(buildname)? "run" : buildname;
string thedate = $"{System.DateTime.Today.Day}-{System.DateTime.Today.Month}-{System.DateTime.Today.Year}";
buildNr = String.IsNullOrEmpty(buildNr)? thedate : buildNr;

string database_name = String.IsNullOrEmpty(dbname)? "DB_Benchmarks" : dbname;
string table_name = String.Concat(buildname, "_", buildNr);

In [ ]:
string CurrentDocDir = Directory.GetCurrentDirectory();
string path = String.Concat(CurrentDocDir,@"/",table_name,".json");
var SesTab = TableExtensions.LoadFromFile(path);

Error: System.IO.FileNotFoundException: Could not find file 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\run_14-1-2022.json'.
File name: 'b:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\run_14-1-2022.json'
   at System.IO.FileStream.ValidateFileHandle(SafeFileHandle fileHandle)
   at System.IO.FileStream.CreateFileOpenHandle(FileMode mode, FileShare share, FileOptions options)
   at System.IO.FileStream..ctor(String path, FileMode mode, FileAccess access, FileShare share, Int32 bufferSize, FileOptions options)
   at System.IO.StreamReader.ValidateArgsAndOpenPath(String path, Encoding encoding, Int32 bufferSize)
   at System.IO.StreamReader..ctor(String path, Encoding encoding, Boolean detectEncodingFromByteOrderMarks)
   at System.IO.File.InternalReadAllText(String path, Encoding encoding)
   at System.IO.File.ReadAllText(String path)
   at BoSSS.Application.BoSSSpad.TableExtensions.LoadFromFile(String filePath) in B:\BoSSS

In [ ]:
//var SesTab = TableExtensions.LoadFromFile(@"B:\BoSSS-smuda\public\doc\handbook\apdx-MPISolverPerformance\DGrotSphere\data_weak\weak_scaling_3_1_2021-9-24.json");

In [ ]:
SesTab.GetColumnNames().Where(s=>s.Contains("Velocity*"))

index,value
0,DGdegree:Velocity*
1,DGdegree:FilteredVelocity*


In [ ]:
var DegreeDict = new Dictionary<int, int>();
var Rows = SesTab.ExtractColumns("DGdegree:Velocity*").Rows;
var Degrees = new List<int>();
foreach(System.Data.DataRow R in Rows){
    int i = 0;
    try{
        int dgr = Convert.ToInt32(R[0]);
        DegreeDict.Add(dgr,i);
        Degrees.Add(dgr);
    }
    catch (Exception ex){

    }
} Degrees

index,value
0,3
1,4
2,2


In [ ]:
// method profile of runs
string[] ColumnNames = new string[]{
"NoOfCores",
"TotalRuntime",
"MatrixAssembly_time",
"AggregationBaseInit_time",
"DataIO_time",
"CGProjection_time",
"SayeCompile_time",
"StandardCompile_time",
"AMR_time",
"LoadBal_time",
"SolverInit_time",
"SolverRun_time"};

In [ ]:
// Filters rows accordin to DgDegree
var ProfileTab = SesTab.ExtractColumns(ColumnNames);
ProfileTab.Clear();
foreach(System.Data.DataRow Row in SesTab.Rows){
    var newrow = ProfileTab.NewRow();
    foreach(string CName in ColumnNames){
        newrow[CName]=Convert.ToDouble(Row[CName]);
    }
    ProfileTab.Rows.Add(newrow);
}

In [ ]:
System.Data.DataView view = new System.Data.DataView(ProfileTab);
view.Sort = "NoOfCores";
ProfileTab=view.ToTable();
ProfileTab.Print()

    NoOfCores TotalRuntime MatrixAssembly_time AggregationBaseInit_time DataIO_time        CGProjection_time SayeCompile_time    StandardCompile_time AMR_time  LoadBal_time SolverInit_time SolverRun_time 
0:  8         138.1026794  15.3548302          0.0086634                1.0141505          0                 3.6631656           0.023946000000000002 0.0020349 1.0807484    4.6684709       103.5947927    
1:  8         69.8270699   7.9216885           0.0095001                1.0594376          0                 1.3265043           0.0232519            0.0020785 0.7179232    2.9038641       49.2069106     
2:  8         44.2824267   6.2549793           0.0314111                1.4156482          0                 0.4496565           0.025666800000000004 0.0021435 0.8397951    1.602707        24.6287368     
3:  16        263.1253955  14.6639407          0.0090544                1.0782527          0                 2.6771899           0.0249197            0.0021269 1.1754913    4.71684

In [ ]:
int L = ProfileTab.Rows.Count();
int xMin = Convert.ToInt32(ProfileTab.ExtractColumns("NoOfCores").Rows[0].ItemArray.First());
int xMax = Convert.ToInt32(ProfileTab.ExtractColumns("NoOfCores").Rows[L-1].ItemArray.First());
new object[]{xMin,xMax}

index,value
0,8
1,512


In [ ]:
// Generate rnd Formatting for massive multiplots
var theDict = new List<Tuple<string,DashTypes,LineColors>>();
public static Tuple<string,DashTypes,LineColors>[] AofFormats;
var dashes = new DashTypes[]{DashTypes.Dashed,DashTypes.DotDashed,DashTypes.DotDotDashed};
var points= new PointTypes[]{PointTypes.OpenCircle,PointTypes.OpenCircle,PointTypes.OpenDiamond,PointTypes.OpenUpperTriangle};
var colors = new LineColors[]{LineColors.Red, LineColors.Green, LineColors.Blue, LineColors.Magenta, LineColors.Yellow, LineColors.Black, LineColors.Orange, LineColors.Grey};

int iSweep=0;
foreach(string Name in ProfileTab.GetColumnNames()){
    //if(Name!="OrthonormalizationMultigrid.MinimizeResidual"&&Name!="overlap_scaling")
    //    continue;
    if(Name=="NoOfCores"||Name=="LSolverIter"||Name=="DGdegree:Velocity*"||Name=="NoOfNodes"||Name=="MGDepth"||Name=="DOF")
        continue;
    int one=iSweep%(dashes.Length);
    int two=iSweep/(dashes.Length);
    Console.WriteLine($"{one}, {two}");
    theDict.Add(new Tuple<string,DashTypes,LineColors>(Name,dashes[one],colors[two]));
    iSweep++;
}
AofFormats = theDict.ToArray()

0, 0
1, 0
2, 0
0, 1
1, 1
2, 1
0, 2
1, 2
2, 2
0, 3
1, 3


index,Item1,Item2,Item3
0,TotalRuntime,Dashed,Red
1,MatrixAssembly_time,DotDashed,Red
2,AggregationBaseInit_time,DotDotDashed,Red
3,DataIO_time,Dashed,Green
4,CGProjection_time,DotDashed,Green
5,SayeCompile_time,DotDotDashed,Green
6,StandardCompile_time,Dashed,Blue
7,AMR_time,DotDashed,Blue
8,LoadBal_time,DotDotDashed,Blue
9,SolverInit_time,Dashed,Magenta


In [ ]:
Func<int, PlotRowSelectorEx> Solver_DG_Proj = delegate (int DG){
    return delegate (int iSweep, int iTabRow, 
                    IDictionary<string, object> Row, 
                    out string Nmn, 
                    out PlotFormat Fmt,
                    out double xValue, out double yValue) { 

            int k  = Convert.ToInt32(Row["DGdegree:Velocity*"]);
            Nmn = AofFormats[iSweep].Item1;

            if(k != DG ) {
                // degree does not match -> not in this plot
                Nmn    = null;
                Fmt    = null;
                xValue = 0;
                yValue = 0;
                return;
            } 
            
            Fmt           = new PlotFormat();
            Fmt.PointSize = 0.5;
            Fmt.Style     = Styles.LinesPoints;
            Fmt.LineWidth = 3;

            Fmt.DashType = AofFormats[iSweep].Item2;
            Fmt.LineColor = AofFormats[iSweep].Item3;
            double iter = Convert.ToDouble(Row["LSolverIter"]);
            yValue = Convert.ToDouble(Row[Nmn])/iter;
            //yValue = Convert.ToDouble(Row[Nmn]);
            xValue    = Convert.ToDouble(Row["NoOfCores"]);
        };
    };

In [ ]:
Func<int, Plot2Ddata[,]> GenDiagram = delegate(int k){
    Plot2Ddata[,] multiplots = new Plot2Ddata[1,1];
    //multiplots[0,0] = SesTab.ToPlot(DoWhatUWant.Solver_DG_Proj(),AofFormats.Length);
    multiplots[0,0] = SesTab.ToPlot(Solver_DG_Proj(k),AofFormats.Length);
    multiplots[0,0].LogX = true;
    multiplots[0,0].LogY = true;
    multiplots[0,0].tmargin = 0;
    multiplots[0,0].bmargin = 5;
    multiplots[0,0].XrangeMin=xMin;
    multiplots[0,0].XrangeMax=xMax;
    multiplots[0,0].YrangeMin=.000001;
    multiplots[0,0].YrangeMax=100000;
    multiplots[0,0].Ylabel = "iter";
    multiplots[0,0].Xlabel = "cores";
    multiplots[0,0].LegendAlignment=new string[]{"i","t","r"};
    multiplots[0,0].LegendHorizontal = true;
    return multiplots;
};

In [ ]:
GenDiagram(2).PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right horizontal Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 TotalRuntime 
 
 
 
 
 TotalRuntime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MatrixAssemblytime 
 
 
 MatrixAssembly t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggregationBaseInittime 
 
 
 AggregationBaseInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DataIOtime 
 
 
 DataIO t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CGProjectiontime 
 
 
 CGProjection t ime 
 
 
 
 
 
 
 SayeCompiletime 
 
 
 SayeCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 StandardCompiletime 
 
 
 StandardCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AMRtime 
 
 
 AMR t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 LoadBaltime 
 
 
 LoadBal t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SolverInittime 
 
 
 SolverInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SolverRuntime 
 
 
 SolverRun t ime

In [ ]:
GenDiagram(3).PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right horizontal Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 TotalRuntime 
 
 
 
 
 TotalRuntime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 MatrixAssemblytime 
 
 
 MatrixAssembly t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AggregationBaseInittime 
 
 
 AggregationBaseInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 DataIOtime 
 
 
 DataIO t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 CGProjectiontime 
 
 
 CGProjection t ime 
 
 
 
 
 
 
 SayeCompiletime 
 
 
 SayeCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 StandardCompiletime 
 
 
 StandardCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 AMRtime 
 
 
 AMR t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 LoadBaltime 
 
 
 LoadBal t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SolverInittime 
 
 
 SolverInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 
 
 SolverRuntime 
 
 
 SolverRun t ime

In [ ]:
GenDiagram(4).PlotNow()

Using gnuplot: B:\BoSSS-BIN\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe
set key font ",16"inside top right horizontal Left reverse 


<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 $10 -6 $ 
 
 
 
 
 $10 -4 $ 
 
 
 
 
 $10 -2 $ 
 
 
 
 
 $10 0 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 $10 4 $ 
 
 
 
 
 $10 1 $ 
 
 
 
 
 $10 2 $ 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 iter 
 
 
 
 
 cores 
 
 
 
 
 TotalRuntime 
 
 
 
 
 TotalRuntime 
 
 
 
 
 
 
 
 
 
 
 
 MatrixAssemblytime 
 
 
 MatrixAssembly t ime 
 
 
 
 
 
 
 
 
 
 
 
 AggregationBaseInittime 
 
 
 AggregationBaseInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 DataIOtime 
 
 
 DataIO t ime 
 
 
 
 
 
 
 
 
 
 
 
 CGProjectiontime 
 
 
 CGProjection t ime 
 
 
 
 
 
 
 SayeCompiletime 
 
 
 SayeCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 StandardCompiletime 
 
 
 StandardCompile t ime 
 
 
 
 
 
 
 
 
 
 
 
 AMRtime 
 
 
 AMR t ime 
 
 
 
 
 
 
 
 
 
 
 
 LoadBaltime 
 
 
 LoadBal t ime 
 
 
 
 
 
 
 
 
 
 
 
 SolverInittime 
 
 
 SolverInit t ime 
 
 
 
 
 
 
 
 
 
 
 
 SolverRuntime 
 
 
 SolverRun t ime

In [ ]:
/*
string dirpath = Directory.GetCurrentDirectory();
ProfileTab.ToCSVFile(dirpath+@"\profile_k"+k+".dat"); dirpath
*/